In [1]:
# Amend to set wd to STB_social_media_analytics.
%cd /home/jia/Desktop/git/STB_social_media_analytics/experimentation/jiaxin_experiment/QY

/home/jia/Desktop/git/STB_social_media_analytics/experimentation/jiaxin_experiment/QY


In [2]:
import pandas as pd
import mysql.connector
from scrapy import Selector
import os
import re
import yaml
import utils
from time import sleep
import traceback
from random import random
from selenium import webdriver
from datetime import datetime, date, timedelta
from random import randint

In [5]:
with open('config_file_QY.yml') as file:
    configs = yaml.load(file, Loader=yaml.FullLoader)

chromedriver_path = configs['General']['chromedriver_path']

db_in_flag = configs['QY']['db_in_flag']
db_out_flag = configs['QY']['db_out_flag']

if db_in_flag == 'csv':
    
    df = pd.read_csv("trialQYlist.csv")
    
    def parse_reviews(text):
        if re.search(r'\d+',str(text)):
            return int(re.search(r'\d+', text).group())
        else:
            return 0

    df['POI'] = df['POI'].apply(lambda x: x.replace(' ',''))    
    df['num_reviews'] = df['num_reviews'].apply(parse_reviews)
    df['link'] = df['link'].apply(lambda x: 'https://'+x[2:])
    
    poi_index=list(range(len(df['POI'])))
    
    poi_df = pd.DataFrame({'poi_index':poi_index, 
                           'poi_name':df['POI'], 
                           'poi_url':df['link']}
                           )
    
    ### FOR POC ONLY ###
    #poi_index = [1, 2, 3, 4, 5]
    #poi_name = ['Gardens by the Bay',                        #2 popular POI, 2 not so popular ones, 1 404 test 
     #           'Marina Bay Sands Hotel and Sands Skypark',
      #          'Thian Hock Keng Temple',
       #         'Arab Street',
        #        '404test'
         #      ]
    #poi_url = ['https://place.qyer.com/poi/V2UJY1FlBzZTZFI3/',
     #          'https://place.qyer.com/poi/V2YJalFkBzBTbA/',
      #         'https://place.qyer.com/poi/V2EJZVFnBz9TYA/',
       #        'https://place.qyer.com/poi/V2AJY1FmBzFTZg/',
        #       'https://place.qyer.com/poi/V2AJY1jjtest/'
         #     ]
    #poi_df = pd.DataFrame({'poi_index':poi_index, 
     #                      'poi_name':poi_name, 
      #                     'poi_url':poi_url}
       #                    )
    ####################

if db_in_flag in ['sqlite', 'mysql']:
    poi_df = pd.DataFrame()

if db_in_flag in ['sqlite', 'mysql'] or db_out_flag in ['sqlite', 'mysql']:
    cnx = mysql.connector.connect(host=configs['General']['host'],
                                  database=configs['General']['database'],
                                  user=configs['General']['user'],
                                  password=configs['General']['password']
                                 )
else:
    cnx = None

In [6]:
class CrawlQY:
    attributes_col_names = ['POI_INDEX',
                            'TOTAL_REVIEWS',
                            'ABOUT_SUMMARY',
                            #'ABOUT',
                            'RANKING',
                            'TICKET_INFO',
                            'RATES',
                            'ADDRESS',
                            'PHONE',
                            'WEBSITE',
                            'TRANSPORTATION_MODE',
                            'OPENING_HOURS',
                            'ATTRIBUTES_CRAWLED_TIME'
                           ]

    reviews_col_names = ['REVIEW_INDEX',
                         'WEBSITE_INDEX',
                         'POI_INDEX',
                         'REVIEWER_URL',
                         'REVIEW_RATING',
                         'REVIEW_DATE',
                         'REVIEW_BODY',
                         'ATTRIBUTES_CRAWLED_TIME'
                         #'TRANSLATED_REVIEW_BODY_GOOGLE',
                         #'TRANSLATED_REVIEW_BODY_WATSON'
                        ]
    
    #reviewers_col_names = ['REVIEWER_URL',
                           #'REVIEWER_NAME',
                           #'REVIEWER_LEVEL',
                           #'GENDER',
                           #'IMAGE_URL',
                           #'CHINESE_RAW_LOCATION',
                           #'CLEANED_ENGLISH_LOCATION',
                           #'NUM_CITIES_VISITED',
                           #'NUM_CTRIES_VISITED',
                           #'REVIEWER_UPDATED_TIME'
                          #]
    
    
    def __init__(self, chromedriver_path, poi_df, cnx, db_out_flag):
        self.driver = webdriver.Chrome(chromedriver_path)
        self.poi_df = poi_df
        if cnx:
            self.cursor = cnx.cursor()
        self.db_out_flag = db_out_flag
        self.number_of_pages = None
        self.current_page = None
        self.current_poi_index = None
        self.review_final_page = False
        self.attributes_df = pd.DataFrame(columns=self.attributes_col_names)
        self.reviews_df = pd.DataFrame(columns=self.reviews_col_names)
        self.sel=None
         
        
        # Create unique CSVs.
        self.datetime_string = datetime.now().strftime('%y%m%d_%H%M%S')
        os.makedirs('./output/{}/'.format(self.datetime_string))
        os.makedirs('./output/{}/reviews'.format(self.datetime_string))
        self.attributes_df.to_csv('./output/{}/attributes.csv'.format(self.datetime_string),mode='a', index=False)
        
        
    
    def add_to_database(self):
        # Read csv, add to database, then cnx.commit().
        return
    
    
    def crawl_pois(self, number_of_pages=None):
        if number_of_pages is not None:
            self.number_of_pages = number_of_pages
        for _, row in self.poi_df.iterrows():
            print('########## {} ##########'.format(row['poi_name']))
        # Create <POI_INDEX>.csv in reviews and reviewers folders.
            self.current_poi_index = row['poi_index']
            self.reviews_df.to_csv('./output/{}/reviews/{}.csv'.format(self.datetime_string, self.current_poi_index), mode='a', index=False)
         
        # Crawl
            try:
                self.driver.get(row['poi_url'])
    #################trial part to handle error of blank pages########################################
                driver=self.driver
                self.sel = Selector(text=driver.page_source)
                sleep(1+random()*2)
                res = self.sel.xpath('//div[@id="app"]')
                if  res == []:
                    log_file = open('./output/{}/log.txt'.format(self.datetime_string), 'a+')
                    log_file.write('{}, {}, {}, {}'.format(row['poi_index'],
                                                           row['poi_name'],
                                                           datetime.now(),
                                                           'POI website not working'))
                    continue
        

    ####################################################################################################
                self.crawl_attributes(row['poi_index'])
                self.attributes_df.to_csv('./output/{}/attributes.csv'.format(self.datetime_string),mode='a', header=False, index=False)
                self.attributes_df = pd.DataFrame(columns=self.attributes_col_names)
                self.crawl_reviews(row['poi_index'])
                if self.db_out_flag != 'csv':
                    self.add_to_database()
                    
            except Exception as e:
                log_file = open('./output/{}/log.txt'.format(self.datetime_string), 'a+')
                log_file.write('{}, {}, page: {}, {}\n'.format(row['poi_index'], row['poi_name'], self.current_page, datetime.now()))
                # log_file.write(repr(e) + '\n')
                log_file.write(traceback.format_exc() + '\n')
                log_file.close()
                break
        
    
    def crawl_reviews(self, poi_index):
        if self.number_of_pages is not None:
            self.current_page = 1                               
            for i in range(self.number_of_pages):
                print('Page {}'.format(self.current_page))
                self.crawl_reviews_1_page(poi_index)
                self.reviews_to_csv()
                self.current_page += 1
                if self.review_final_page:
                    self.review_final_page= False
                    break
        else:
            self.current_page = 1
            while not self.review_final_page:
                print('Page {}'.format(self.current_page))
                self.crawl_reviews_1_page(poi_index)
                self.reviews_to_csv()
                self.current_page += 1
                if self.review_final_page:
                    self.review_final_page= False
                    break
                    
                    
    def reviews_to_csv(self):
        self.reviews_df.to_csv('./output/{}/reviews/{}.csv'.format(self.datetime_string, self.current_poi_index), mode='a', header=False, index=False)
        self.reviews_df = pd.DataFrame(columns=self.reviews_col_names)
    
    def crawl_attributes(self, poi_index):
      
        #select areas where we going to extract info
        res = self.sel.xpath('.//div[@class="poi-detail"]')    
        
        
        #need parse ,"xxx条点评"
        xpath_tot_reviews = '//a[@data-bn-ipg="place-poi-detail-viewAllReviews"]/text()'     
        total_review=res.xpath(xpath_tot_reviews).extract_first() 

        xpath_summary = '//div[@class="poi-detail"]/div/p/text()'
        if res.xpath(xpath_summary).extract_first() != None:
            about_summary=res.xpath(xpath_summary).extract_first() 
        else: 
            about_summary='Not Available'
        
        #about is missing here, need to clarify the difference between about and about_summary
        
        
        #need parse, "第x名",change to int
        xpath_ranking = '//li[@class="rank"]/span/text()'     
        ranking_bf=res.xpath(xpath_ranking).extract_first() 
        
        xpath_ticket = './/li[./span[contains(text(),"门票：")]]/div/p/text()'
        if res.xpath(xpath_ticket).extract_first() != None:
            ticket_info=res.xpath(xpath_ticket).extract_first()
        else:
            ticket_info = str(None)

        #parse, remove spaces, change to float
        xpath_rate = '//span[@class="number"]/text()'    
        rate=res.xpath(xpath_rate).extract_first()
        
        xpath_address='.//li[./span[contains(text(),"地址：")]]/div/p/text()'
        if res.xpath(xpath_address).extract_first() != None:
            address=res.xpath(xpath_address).extract_first() 
        else:
            address='Not Available'
            
        xpath_phone='.//li[./span[contains(text(),"电话：")]]/div/p/text()'
        if res.xpath(xpath_phone).extract_first() != None:
            phone=res.xpath(xpath_phone).extract_first() 
        else:
            phone='Not Available'


        xpath_web =  './/li[./span[contains(text(),"网址：")]]/div/a/@href'
        if res.xpath(xpath_web).extract_first()!=None:
            website=res.xpath(xpath_web).extract_first() 
        else:
            website='Not Available'

        #need edit, share same xpath with phone case 2
        xpath_trans = './/li[./span[contains(text(),"到达方式：")]]/div/p/text()'  
        if res.xpath(xpath_trans).extract_first() != None :
            transportation_mode=res.xpath(xpath_trans).extract_first() 
        else:
            transportation_mode='Not Available'
  

        xpath_openhr = './/li[./span[contains(text(),"开放时间：")]]/div/p/text()'
        if res.xpath(xpath_openhr).extract_first() != None:
            opening_hours=res.xpath(xpath_openhr).extract_first()  
        else:
            opening_hours='Not Available'
        
        # Parsing attributes.
        total_reviews=self.parse_total_reviews(total_review)
        ranking=self.parse_ranking(ranking_bf)
        rates=self.parse_rate(rate)
        
        poi_attributes = [poi_index,
                          total_reviews, 
                          about_summary, 
                          #about, 
                          ranking,
                          ticket_info,
                          rates,
                          address,
                          phone,
                          website,
                          transportation_mode,
                          opening_hours,
                          datetime.now()
                         ]
        
        # Inserting attributes into dataframe
        poi_attributes_dict = dict(zip(self.attributes_col_names, poi_attributes))
        self.attributes_df = self.attributes_df.append(poi_attributes_dict, ignore_index=True)

    def crawl_reviews_1_page(self, poi_index):        
        driver=self.driver
        self.sel = Selector(text=driver.page_source)
        res1 = self.sel.xpath('.//div[@class="compo-main compo-feed"]') 
        sleep(random()*2)
        
        xpath_url = './/a[@class="largeavatar"]/@href' 
        reviewer_url = res1.xpath(xpath_url).getall()   
        sleep(1+random()*2)

        xpath_rates = './/ul[@class="comment-list"]//li//span[@class="poi-stars"]'
        star = res1.xpath(xpath_rates).getall()
        def regex_cnt(string,pattern):
            return len(re.findall(pattern,string))
        review_rating=list(map(lambda x:regex_cnt(x,"singlestar full"),star))
       

        xpath_date = './/a[@class="date"]/text()'  
        review_date1 = res1.xpath(xpath_date).getall()   

        xpath_body= './/p[@class="content"]/text()'  
        review_body1= res1.xpath(xpath_body).getall()
        
        
        
        # Parsing reviews
        review_body=list(map(self.parse_review_body,review_body1))
        review_date=list(map(self.parse_review_date,review_date1))
            
        review_details = [None, # REVIEW_INDEX
                          4, # WEBSITE_INDEX (QY is '4')
                          poi_index,
                          reviewer_url,
                          review_rating,
                          review_date,
                          review_body,
                          #translated_review_body_google,
                          #translated_review_body_watson,
                          datetime.now()
                          ]
            
        #reviewer_details = [reviewer_url,
                            #reviewer_name,
                            #reviewer_level,
                            #gender,
                            #image_url,
                            #chinese_raw_location,
                            #cleaned-english_location,
                            #num_cities_visited,
                            #num_ctries_visited,
                            #eviewer_update_time
                            #datetime.now()
                            #]
            
        # Inserting reviews into dataframe.
        review_details_dict = dict(zip(self.reviews_col_names, review_details))
        self.reviews_df = self.reviews_df.append(review_details_dict, ignore_index=True)
            
        # Inserting reviewers into dataframe.
        #reviewer_details_dict = dict(zip(self.reviewers_col_names, reviewer_details))
        #self.reviewers_df = self.reviewers_df.append(reviewer_details_dict, ignore_index=True)

        next_page_button = self.driver.find_elements_by_xpath('//a[@title="下一页"]')
        if next_page_button:
            sleep(1+random()*2)
            self.driver.execute_script("arguments[0].click()",next_page_button[0])
            sleep(1+random()*2)
            self.sel = Selector(text=self.driver.page_source)
        else :
            print("Can't click or last page")
            self.review_final_page= True
         
    
    # Methods below are all utility functions.
    def parse_total_reviews(self, text):
        if re.search(r'\d+',str(text)):
            return int(re.search(r'\d+', text).group())
        else:
            return 0
    
    def parse_ranking(self, text):
        return int(re.search(r'\d+', text).group())
    
    def parse_rate(self, text):
        if text:
            return float(text.replace(' ', ''))
        else:
            return 0
        
    def parse_review_date(self, text):
        text1=text.replace('\n                  ','')
        return datetime.strptime(text1[0:10],"%Y-%m-%d")
    
    def parse_review_body(self, text):
        return text.replace('\n','')
    
    

In [7]:
CrawlQY(chromedriver_path, poi_df, cnx, db_out_flag).crawl_pois(number_of_pages=3)

########## 
滨海湾金沙酒店与金沙空中花园
  
 ##########
Page 1
Page 2
Page 3
########## 
圣淘沙岛
  
 ##########
Page 1
Page 2
Page 3
########## 
新加坡环球影城
  
 ##########
Page 1
Page 2
Page 3
########## 
滨海湾花园
  
 ##########
Page 1
Page 2
Page 3
########## 
圣淘沙名胜世界
  
 ##########
Page 1
Page 2
Page 3
########## 
新加坡摩天观景轮
  
 ##########
Page 1
Page 2
Page 3
########## 
克拉码头
  
 ##########
Page 1
Page 2
Page 3
########## 
新加坡动物园
  
 ##########
Page 1
Page 2
Page 3
########## 
鱼尾狮公园
  
 ##########
Page 1
Page 2
Page 3
########## 
滨海湾花园超级树
  
 ##########
Page 1
Page 2
Page 3
########## 
乌节路
  
 ##########
Page 1
Page 2
Page 3
########## 
新加坡植物园
  
 ##########
Page 1
Page 2
Page 3
########## 
莱佛士酒店
  
 ##########
Page 1
Page 2
Page 3
########## 
圣淘沙鱼尾狮塔
  
 ##########
Page 1
Page 2
Page 3
########## 
滨海艺术中心
  
 ##########
Page 1
Page 2
Page 3
########## 新加坡夜间野生动物园   ##########
Page 1
Page 2
Page 3
########## 新加坡河   ##########
Page 1
Page 2
Page 3
########## 螺旋桥   ##########
Page 1
Page 2
Page 3
########## 牛车水原

Page 1
Can't click or last page
########## 新加坡国立大学柏林墙   ##########
Page 1
Can't click or last page
########## 新加坡科技与设计大学   ##########
Page 1
Can't click or last page
########## 新加坡艺术学院   ##########
Page 1
Can't click or last page
########## 新加坡圣若瑟堂   ##########
Page 1
Can't click or last page
########## 莲山双林寺   ##########
Page 1
Can't click or last page
########## 圣詹姆斯发电厂   ##########
Page 1
Can't click or last page
########## 科尼岛公园   ##########
Page 1
Can't click or last page
########## 新加坡西海岸公园   ##########
Page 1
Page 2
Can't click or last page
########## 亚历山大拱桥   ##########
Page 1
Can't click or last page
########## 达豪施纪念碑   ##########
Page 1
Can't click or last page
########## 拉萨尔设计学院   ##########
Page 1
Can't click or last page
########## 新加坡军事博物馆   ##########
Page 1
Can't click or last page
########## 新加坡基督教青年会   ##########
Page 1
Can't click or last page
########## 鸦片山岗战役纪念馆   ##########
Page 1
Can't click or last page
########## 峇峇住宅   ##########
Page 1
Can't click or last pag

Can't click or last page
########## 纳哥德卡回教圣殿   ##########
Page 1
Can't click or last page
########## LightOfChristCityChurch   ##########
Page 1
Can't click or last page
########## SingaporeLifeChurch   ##########
Page 1
Can't click or last page
########## FoochowMethodistChurch   ##########
Page 1
Can't click or last page
########## KampongKaporMethodistChurch   ##########
Page 1
Can't click or last page
########## ChurchOftheTrueLight   ##########
Page 1
Can't click or last page
########## LighthouseBaptistChurch   ##########
Page 1
Can't click or last page
########## 莆田天道堂   ##########
Page 1
Can't click or last page
########## ElimChurchAssemblyOfGod   ##########
Page 1
Can't click or last page
########## St.Paul'sChurch   ##########
Page 1
Can't click or last page
########## SerangoonCatholicChurchStJosephDyingAidAssn   ##########
Page 1
Can't click or last page
########## TokyoPastaMario(威士马广场店)   ##########
Page 1
Can't click or last page
########## GraceSingaporeChineseChristia